<a href="https://colab.research.google.com/github/zy4kamu/FireMisha/blob/main/intent_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Install HuggingFace transformers

!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ptk6g4gj
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-ptk6g4gj
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-3.3.1-cp36-none-any.whl size=1082350 sha256=a7090affd8307221f1eedda5024dc0f1bcb713cd21e89ede7df73e83313e546e
  Stored in directory: /tmp/pip-ephem-wheel-cache-mgp9wzzi/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
Successfully built transformers


In [10]:
# Download train and validate datasets
!rm -rf /tmp/classification_train
!rm -rf /tmp/classification_validate
!wget https://raw.githubusercontent.com/zy4kamu/FireMisha/main/classification_train -O /tmp/classification_train
!wget https://raw.githubusercontent.com/zy4kamu/FireMisha/main/classification_validate -O /tmp/classification_validate

train_file = '/tmp/classification_train'
validate_file = '/tmp/classification_validate'

--2020-10-03 19:11:13--  https://raw.githubusercontent.com/zy4kamu/FireMisha/main/classification_train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1010944 (987K) [text/plain]
Saving to: ‘/tmp/classification_train’

/tmp/classification 100%[===================>] 987.25K  --.-KB/s    in 0.06s   

2020-10-03 19:11:13 (15.3 MB/s) - ‘/tmp/classification_train’ saved [1010944/1010944]

--2020-10-03 19:11:14--  https://raw.githubusercontent.com/zy4kamu/FireMisha/main/classification_validate
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51981

In [11]:
# Import required packages

from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

import tensorflow as tf
import json

In [12]:
# Read training and validate datasets

def read_dataset(input_file):
  sentences = []
  labels = []
  with open(input_file) as reader:
    train_json = json.load(reader)
    for item in train_json:
      sentences.append(item['text'])
      labels.append(item['label'])
  return sentences, labels 

training_sentences, training_labels = read_dataset(train_file)
validation_sentences, validation_labels = read_dataset(validate_file)

print('Training sentences:', training_sentences[0:3], '...')
print('Training labels:', training_labels[0:3], '...')
print('Validation sentences:', validation_sentences[0:3], '...')
print('Validation labels:', validation_labels[0:3], '...')

Training sentences: ['play a jack lawrence concerto', 'play some sam moore.', 'add album to princesas indie'] ...
Training labels: [6, 6, 5] ...
Validation sentences: ['where is belgium located', 'let me hear the good songs from james iha', 'play something by duke ellington from the seventies'] ...
Validation labels: [0, 6, 6] ...


In [13]:
# Create training and validation datasets for TensorFlow backend

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

In [ ]:
# Train the model

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                              num_labels=7)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))
model.save_pretrained('/tmp/classification_model')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/3
434/862 [==============>...............] - ETA: 52s - loss: 0.2314 - accuracy: 0.9515

In [ ]:
# Load model and make prediction on specific sentence 

loaded_model = TFDistilBertForSequenceClassification.from_pretrained('/tmp/classification_model')
test_sentence = 'play a jack lawrence concerto'

predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors='tf')

tf_output = tf.nn.softmax(loaded_model.predict(predict_input)[0]).numpy()[0]

labels = ['GetWeather', 'SearchCreativeWork', 'RateBook', 'SearchScreeningEvent',
          'BookRestaurant', 'AddToPlaylist', 'PlayMusic']
print()
for label, probability in zip(labels, tf_output):
  print('{}: {:0.5f}'.format(label, probability))

Some weights of the model checkpoint at /tmp/classification_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_99']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /tmp/classification_model and are newly initialized: ['dropout_359']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



GetWeather: 0.00004
SearchCreativeWork: 0.00052
RateBook: 0.00004
SearchScreeningEvent: 0.00008
BookRestaurant: 0.00008
AddToPlaylist: 0.00015
PlayMusic: 0.99909
